## Image Caption Generator

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.datasets import CocoDetection
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import os
from PIL import Image
import numpy as np
import requests
from io import BytesIO
from datasets import load_dataset
from tqdm import tqdm
import cv2
import pycocoevalcap
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from collections import Counter
# for metrics, COCO API metrics is used
import json
from pycocoevalcap import bleu, meteor, rouge, spice
#from pycocoevalcap.evalcap import COCOEvalCap

# Load YOLOv5 for feature extraction
import torch.hub

In [3]:
# setting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load pre-trained YOLOv5 model from Torch Hub
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).eval().to(device)

Using cache found in /home/pokepe/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-17 Python-3.10.15 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
# Define transformations for image preprocessing
# !!!!! NOTE chatgpt says this is not needed in yolov5
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Vocabulary, will be used later.
vocabulary = []

In [7]:
# "annotations": [{"image_id": 179765,"id": 38,"caption": "A black Honda motorcycle parked in front of a garage."},...}
class CocoDataset(Dataset): # <start> cat sat on the mat <end> -> {0 32 24 34 3 3 1 }
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.tokenizer = get_tokenizer("basic_english")  # Tokenizer from torchtext
        self.annotations = self.coco_data['annotations']
        # Build vocabulary
        self.vocab = self.build_vocab()
        self.image_id_to_file = {img['id']: img['file_name'] for img in self.coco_data['images']}
        self.image_id_to_img = {}


    def build_vocab(self):
        counter = Counter()
        for annotation in tqdm(self.annotations):
            caption = annotation['caption']
            tokens = self.tokenizer(caption.lower())
            counter.update(tokens)
        vocab = build_vocab_from_iterator([counter], specials=["<pad>", "<unk>", "<bos>", "<eos>"])
        vocab.set_default_index(vocab["<unk>"])  # Out-of-vocabulary words will be mapped to <unk>
        global vocabulary
        vocabulary = vocab
        return vocab

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_id = self.annotations[idx]['image_id']

        file_name = self.image_id_to_file.get(image_id)
        img_path = os.path.join(self.root_dir, file_name)

        try:
          image = Image.open(img_path).convert('RGB')
          self.image_id_to_img[image_id] = image # NOTE
        except Exception as e:
          print(f"Error opening image: {file_name}")
          raise e

        caption = self.annotations[idx]['caption']
        tokens = self.tokenizer(caption.lower())

        # Convert caption tokens to indices
        caption_indices = [self.vocab['<bos>']] + [self.vocab[token] for token in tokens] + [self.vocab['<eos>']]

        if self.transform:
            image = self.transform(image)

        return image, {'image_id': image_id,'captions': torch.tensor(caption_indices, dtype=torch.long)}


In [8]:
#MAINNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.cnn = models.resnet50(pretrained=True).to(device)
        self.backbone = nn.Sequential(*list(self.cnn.children())[:-1])  # Get everything except the last detection layer
        self.fc = nn.Linear(self.cnn.fc.in_features, embed_size)  # Linear layer to resize

    def forward(self, images):
        with torch.no_grad():  # Freeze CNN parameters
            features = self.backbone(images)
        features = features.view(features.size(0), -1)  # Flatten
        features = self.fc(features)
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        embeddings = self.embedding(captions)
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        lstm_out, _ = self.lstm(inputs)
        output = self.fc(lstm_out)
        return output

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.cnn = models.resnet50(pretrained=True).to(device)
        self.backbone = nn.Sequential(*list(self.cnn.children())[:-1])  # Get everything except the last detection layer
        self.fc = nn.Linear(self.cnn.fc.in_features, embed_size)  # Linear layer to resize
        self.dropout = nn.Dropout(0.3)  # Add dropout to the encoder

    def forward(self, images):
        with torch.no_grad():  # Freeze CNN parameters
            features = self.backbone(images)
        features = features.view(features.size(0), -1)  # Flatten
        features = self.fc(features)
        features = self.dropout(features)  # Apply dropout to the feature vector
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, dropout_prob=0.3):
        super(DecoderRNN, self).__init__()
        # Embedding layer to convert words to a vector representation
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Bidirectional LSTM with dropout
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        # Linear layer: multiply hidden_size by 2 because of the bidirectional LSTM
        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        # Add a dropout layer after the LSTM output before the final linear layer
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, features, captions):
        # Convert captions to embeddings
        embeddings = self.embedding(captions)
        # Concatenate features and embeddings along the sequence dimension
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        # Pass through LSTM
        lstm_out, _ = self.lstm(inputs)
        # Apply dropout after LSTM output
        lstm_out = self.dropout(lstm_out)
        # Pass through fully connected layer to generate vocabulary scores
        output = self.fc(lstm_out)
        return output

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


In [9]:
from torch.utils.data import Subset
import random 

train_image_number = 20000
val_image_number = 4000

train_images_path = './coco/train2017'
train_annotations_path = './coco/annotations/captions_train2017.json'
val_images_path = './coco/val2017'
val_annotations_path = './coco/annotations/captions_val2017.json'

# Load COCO Dataset (captions and images) using
train_dataset = CocoDataset(train_images_path, train_annotations_path, transform=transform)

# Example: Fetch an image and its caption
image, caption = train_dataset[0]
print(image)  # Prints the caption token indices

val_dataset = CocoDataset(val_images_path, val_annotations_path, transform=transform)

subset_indices = [i for i in range(4000)]

train_subset = Subset(train_dataset, subset_indices)
val_subset = Subset(val_dataset, subset_indices)


100%|██████████| 591753/591753 [00:03<00:00, 185451.68it/s]


tensor([[[2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         ...,
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891],
         [2.24891, 2.24891, 2.24891,  ..., 2.24891, 2.24891, 2.24891]],

        [[2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         ...,
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857],
         [2.42857, 2.42857, 2.42857,  ..., 2.42857, 2.42857, 2.42857]],

        [[2.64000, 2.64000, 2.64000,  ..., 2.64000, 2.64000, 2.64000],
         [2.64000, 2.64000, 2.64000,  ..., 2.

100%|██████████| 25014/25014 [00:00<00:00, 187834.58it/s]


In [10]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    """Collates a batch of (image, caption) pairs and pads captions."""
    images, targets = zip(*batch)
    image_ids = [target["image_id"] for target in targets]
    captions = [target["captions"] for target in targets]
    # Pad captions to have the same length in the batch
    padded_captions = pad_sequence(captions, batch_first=True, padding_value=0)  # <pad> token is 0

    # Stack images into a batch
    images = torch.stack(images, 0) # dim: The dimension along which to stack. If dim=0, it adds a new first dimension to the tensors.
   # NO NEED FOR padded_captions = torch.stack(padded_captions, 0) , padded_captions are already stacked

    return images, image_ids, padded_captions

In [11]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    for images, image_ids, captions in dataloader:
        #captions = targets['captions']
        images = images.to(device)
        captions = captions.to(device)
        #print(len(captions))
        #print(len(images))
        optimizer.zero_grad()
        outputs = model(images, captions[:, :-1])  # Exclude last token in captions

        batch_size, seq_len, vocab_size = outputs.size()
        outputs = outputs.view(-1, vocab_size)  # Flattens outputs
        captions = captions.view(-1)
        loss = criterion(outputs, captions)
        loss.backward()
        optimizer.step()
        # Print the current loss for monitoring the training progress
        print(f"Loss: {loss.item()}")

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, image_ids, captions in dataloader:
            #captions = targets['captions']
            images = images.to(device)
            captions = captions.to(device)

            #TODO DIMENSIONALITY PROBLEM
            outputs = model(images, captions[:, :-1])  # Exclude last token in captions
            batch_size, seq_len, vocab_size = outputs.size()
            outputs = outputs.view(-1, vocab_size)  # Flattens outputs
            captions = captions.view(-1)
            loss = criterion(outputs, captions)  # Teacher forcing
            total_loss += loss.item()

    return total_loss / len(dataloader)


In [12]:
# create a directory for checkpoints -> this can be used for storing state of the model in the midst of the training.
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok= True)

def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")


In [13]:
# Hyperparameters
embed_size = 256
hidden_size = 512
num_epochs = 20
learning_rate = 0.001
batch_size = 32
num_layers = 1

# Data Loaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Vocab
train_dataset.vocab = train_dataset.build_vocab()
# Instantiate Encoder, Decoder, and Model
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, len(train_dataset.vocab), num_layers=num_layers).to(device) # TODO
model = ImageCaptioningModel(encoder, decoder).to(device)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()



100%|██████████| 591753/591753 [00:03<00:00, 180057.09it/s]


In [14]:
# Training Loop
for epoch in tqdm(range(num_epochs)):
    train(model, train_loader, optimizer, criterion, device)

    print(f"Training for epoch {epoch+1}/{num_epochs} completed.")

    # Commented out - quicker
    #val_loss = evaluate(model, val_loader, criterion, device)
    #print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}")

# Save the trained model
torch.save(model.state_dict(), 'image_captioning_model.pth')

  0%|          | 0/20 [00:00<?, ?it/s]

Loss: 10.303236961364746
Loss: 9.765588760375977
Loss: 9.63361644744873
Loss: 8.469826698303223
Loss: 8.934879302978516
Loss: 8.609504699707031
Loss: 7.357395648956299
Loss: 3.458204507827759
Loss: 5.4030303955078125
Loss: 5.59335470199585
Loss: 3.741607666015625
Loss: 4.528093338012695
Loss: 3.055696964263916
Loss: 3.767695903778076
Loss: 4.262363910675049
Loss: 3.697359085083008
Loss: 3.659416913986206
Loss: 3.507753610610962
Loss: 3.8023314476013184
Loss: 2.6519644260406494
Loss: 2.4620091915130615
Loss: 3.209965944290161
Loss: 2.9807045459747314
Loss: 2.321748733520508
Loss: 3.3059980869293213
Loss: 3.7758474349975586
Loss: 3.577812910079956
Loss: 3.366856098175049
Loss: 2.9022982120513916
Loss: 3.3959171772003174
Loss: 2.109205722808838
Loss: 3.089293956756592
Loss: 3.2200818061828613
Loss: 2.431415319442749
Loss: 3.783979892730713
Loss: 3.1795654296875
Loss: 3.107797145843506
Loss: 3.337613344192505
Loss: 3.935781955718994
Loss: 3.344294548034668
Loss: 2.5701756477355957
Loss: 2.

  5%|▌         | 1/20 [00:57<18:17, 57.74s/it]

Loss: 2.903203248977661
Training for epoch 1/20 completed.
Loss: 2.283644199371338
Loss: 2.926236391067505
Loss: 1.938520908355713
Loss: 2.3469133377075195
Loss: 2.7200067043304443
Loss: 2.664088249206543
Loss: 2.7759318351745605
Loss: 2.4772658348083496
Loss: 2.664966583251953
Loss: 2.2136499881744385
Loss: 1.8992018699645996
Loss: 2.3496780395507812
Loss: 2.274320363998413
Loss: 2.446026086807251
Loss: 2.324126958847046
Loss: 2.1978394985198975
Loss: 2.5261402130126953
Loss: 2.349794626235962
Loss: 2.6232335567474365
Loss: 2.4349141120910645
Loss: 1.8419959545135498
Loss: 2.1446473598480225
Loss: 2.6375203132629395
Loss: 2.434880495071411
Loss: 2.520979166030884
Loss: 2.477773666381836
Loss: 2.5511157512664795
Loss: 1.643232822418213
Loss: 2.7238476276397705
Loss: 2.4706547260284424
Loss: 2.8832101821899414
Loss: 2.2659056186676025
Loss: 2.6295204162597656
Loss: 2.3754796981811523
Loss: 2.525576591491699
Loss: 2.821744441986084
Loss: 2.236917734146118
Loss: 2.3979809284210205
Loss: 2

 10%|█         | 2/20 [01:54<17:08, 57.12s/it]

...

Loss: 0.4487525224685669
Training for epoch 18/20 completed.
Loss: 0.4672106206417084
Loss: 0.5506153702735901
Loss: 0.3618931770324707
Loss: 0.28554290533065796
Loss: 0.5415396094322205
Loss: 0.45187491178512573
Loss: 0.5154229998588562
Loss: 0.41303393244743347
Loss: 0.5730336904525757
Loss: 0.4838342070579529
Loss: 0.49102285504341125
Loss: 0.47674936056137085
Loss: 0.5744675397872925
Loss: 0.4146616458892822
Loss: 0.5081989169120789
Loss: 0.4972776174545288
Loss: 0.5610988140106201
Loss: 0.528190016746521
Loss: 0.5436177849769592
Loss: 0.40899458527565
Loss: 0.47426751255989075
Loss: 0.43283215165138245
Loss: 0.35027503967285156
Loss: 0.519888699054718
Loss: 0.5689070820808411
Loss: 0.40842950344085693
Loss: 0.43705084919929504
Loss: 0.4790489971637726
Loss: 0.482715904712677
Loss: 0.4859209954738617
Loss: 0.4964331090450287
Loss: 0.5019394755363464
Loss: 0.32184457778930664
Loss: 0.4433518350124359
Loss: 0.4760717451572418
Loss: 0.5729319453239441
Loss: 0.558347761631012
Loss: 0.5

 95%|█████████▌| 19/20 [18:09<00:57, 57.41s/it]

Loss: 0.6126670241355896
Training for epoch 19/20 completed.
Loss: 0.5819835066795349
Loss: 0.3393446207046509
Loss: 0.4014557898044586
Loss: 0.48826828598976135
Loss: 0.3481905162334442
Loss: 0.5199215412139893
Loss: 0.3674403429031372
Loss: 0.4801744520664215
Loss: 0.4589276909828186
Loss: 0.40401801466941833
Loss: 0.4250420033931732
Loss: 0.4739600718021393
Loss: 0.42321306467056274
Loss: 0.4779113531112671
Loss: 0.4708516597747803
Loss: 0.4043266177177429
Loss: 0.45678529143333435
Loss: 0.3687330484390259
Loss: 0.487093061208725
Loss: 0.38777753710746765
Loss: 0.45918211340904236
Loss: 0.4636353552341461
Loss: 0.5059917569160461
Loss: 0.4799322485923767
Loss: 0.4955994486808777
Loss: 0.4503340721130371
Loss: 0.4498043656349182
Loss: 0.41796061396598816
Loss: 0.4137883484363556
Loss: 0.37119805812835693
Loss: 0.38375282287597656
Loss: 0.5257906913757324
Loss: 0.2793300747871399
Loss: 0.3580341041088104
Loss: 0.3538447916507721
Loss: 0.4970168471336365
Loss: 0.4973238408565521
Loss: 

100%|██████████| 20/20 [19:07<00:00, 57.35s/it]

Loss: 0.5843541622161865
Training for epoch 20/20 completed.


In [15]:
def generate_caption(image, model, vocab, device):

    model.eval()
    print(image.shape)
    image = image.unsqueeze(0).to(device) # unsqueezing for encapsulating it inside a batch
    print(image.shape)
    features = model.encoder(image)

    # Generate caption
    caption = ['<bos>']
    for _ in range(50):  # Maximum caption length
        input_caption = torch.tensor([vocab[token] for token in caption]).unsqueeze(0).to(device)
        outputs = model.decoder(features, input_caption)
        _, predicted = outputs.max(2)  # outputs.shape = (batch,seq_len,vocab_size), vocab_size needed because we need to calculate each word's probabilty and pick the best of them
        predicted_word = vocab.get_itos()[predicted[0, -1].item()] # get the first batch (only one), last predicted word in lstm(next word) convert it to string instead of an id
        caption.append(predicted_word)
        if predicted_word == '<eos>':
            break
    print(caption)
    return ' '.join(caption[1:-1])  # Removing <bos> and <eos>

# Load trained model and generate a caption
model.load_state_dict(torch.load('image_captioning_model.pth'))
#image = Image.open("path_to_image.jpg")  # Replace with your image path
#caption = generate_caption(image, model, train_dataset.vocab, device)
#print(f"Generated Caption: {caption}")


<All keys matched successfully>

In [16]:
def generate_captions_for_coco(val_loader,model,vocab, max_length= 50, num_samples=5):
    all_captions = {}
    model.eval()
    with torch.no_grad():
        for i, (images, image_ids,captions) in enumerate(val_loader):
            #image_ids = targets['image_id']
            images = images.to(device)
            for image_id, image in zip(image_ids,images):
              generated_caption = generate_caption(image, model, vocab, device)
              print(f"type(generated_caption) = {type(generated_caption)} , {type(image_id)}")
              all_captions[image_id] = generated_caption
              print(f"Generated Caption for Image {image_id}: {generated_caption}")
            if i >= num_samples:
                break
    print(all_captions)
    return all_captions



In [17]:

def evaluate_captioning_model(generated_captions, coco_annotation_file=val_annotations_path,coco_image_dir=val_images_path):
    """
    Evaluate the image captioning model using COCO evaluation metrics: BLEU, METEOR, ROUGE, CIDEr.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.

    Returns:
        dict: Dictionary containing BLEU, METEOR, ROUGE, CIDEr scores.
    """
    # Load the COCO dataset annotations (reference captions)
    coco = COCO(coco_annotation_file)

    # Create a dictionary for the generated captions (image_id -> caption)
    coco_results = [{'image_id': image_id, 'caption': caption} for image_id, caption in generated_captions.items()]


    # Save the generated captions in a temporary file
    with open('generated_captions.json', 'w') as f:
        json.dump(coco_results, f)

    # Load the results into COCO's evaluation API
    coco_results = coco.loadRes('generated_captions.json')
    print(coco_results)
    print("Generated Captions Image IDs:", generated_captions.keys())
    print("Ground Truth Image IDs:", coco.getImgIds())


    # since we filtered the images to contain only first 1000 images, lets filter the metric
    all_image_ids = coco.getImgIds()
    filtered_image_ids = [image_id for image_id in generated_captions.keys()]

    # we need to revise the filtered version of the actual annotations
    gts = {}
    for image_id in filtered_image_ids:
        caption_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(caption_ids)
        gts[image_id] = [annotation['caption'] for annotation in annotations]


    # Set up the evaluation
    print("gts.keys(): ",gts.keys())
    print("coco_results.keys(): ", generated_captions.keys())
    #assert(gts.keys() == coco_results.keys())

    coco_eval = COCOEvalCap(coco, coco_results)
    coco_eval.params['image_id'] = filtered_image_ids
    coco_eval.evaluate()

    # Extract and return the metrics (BLEU, METEOR, ROUGE, CIDEr)
    metrics = coco_eval.eval
    return metrics

generated_captions = generate_captions_for_coco(val_loader,model,vocabulary)
# Evaluate the generated captions
metrics = evaluate_captioning_model(generated_captions)


print("Evaluation Metrics:", metrics)

torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'motorcycle', 'parked', 'in', 'a', 'parking', 'lot', 'with', 'a', 'helmet', 'on', 'the', 'seat', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: a motorcycle parked in a parking lot with a helmet on the seat .
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'motorcycle', 'parked', 'in', 'a', 'parking', 'lot', 'with', 'a', 'helmet', 'on', 'the', 'seat', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: a motorcycle parked in a parking lot with a helmet on the seat .
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'large', 'black', 'cat', 'sits', 'inside', 'of', 'an', 'empty', 'bathtub', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 190236: a large black cat sits inside of an empty bathtub .
torch.Size([3, 224, 2

PTBTokenizer tokenized 2696 tokens at 39480.90 tokens per second.
PTBTokenizer tokenized 576 tokens at 10368.56 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 498, 'reflen': 469, 'guess': [498, 453, 408, 363], 'correct': [227, 71, 14, 6]}
ratio: 1.0618336886970963
Bleu_1: 0.456
Bleu_2: 0.267
Bleu_3: 0.135
Bleu_4: 0.080
computing METEOR score...
METEOR: 0.131
computing Rouge score...
ROUGE_L: 0.350
computing CIDEr score...
CIDEr: 0.262
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.9

SPICE evaluation took: 7.512 s
SPICE: 0.056
Evaluation Metrics: {'Bleu_1': 0.45582329317177545, 'Bleu_2': 0.26728730404406575, 'Bleu_3': 0.13483671952061507, 'Bleu_4': 0.07978427257672112, 'METEOR': 0.13071145719065927, 'ROUGE_L': 0.34994471888266043, 'CIDEr': 0.2622720325163794, 'SPICE': 0.05623738523704123}


In [18]:
import matplotlib.pyplot as plt
from PIL import Image
from pycocotools.coco import COCO

def evaluate_captioning_model(generated_captions, coco_annotation_file=val_annotations_path, coco_image_dir=val_images_path):
    """
    Evaluate the image captioning model using COCO evaluation metrics: BLEU, METEOR, ROUGE, CIDEr.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.

    Returns:
        dict: Dictionary containing BLEU, METEOR, ROUGE, CIDEr scores.
    """
    # Load the COCO dataset annotations (reference captions)
    coco = COCO(coco_annotation_file)

    # Create a dictionary for the generated captions (image_id -> caption)
    coco_results = [{'image_id': image_id, 'caption': caption} for image_id, caption in generated_captions.items()]

    # Save the generated captions in a temporary file
    with open('generated_captions.json', 'w') as f:
        json.dump(coco_results, f)

    # Load the results into COCO's evaluation API
    coco_results = coco.loadRes('generated_captions.json')
    print(coco_results)
    print("Generated Captions Image IDs:", generated_captions.keys())
    print("Ground Truth Image IDs:", coco.getImgIds())

    # Filter the image IDs for evaluation
    filtered_image_ids = [image_id for image_id in generated_captions.keys()]

    # Get the reference annotations for evaluation
    gts = {}
    for image_id in filtered_image_ids:
        caption_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(caption_ids)
        gts[image_id] = [annotation['caption'] for annotation in annotations]

    # Set up the evaluation
    print("gts.keys(): ", gts.keys())
    print("coco_results.keys(): ", generated_captions.keys())

    coco_eval = COCOEvalCap(coco, coco_results)
    coco_eval.params['image_id'] = filtered_image_ids
    coco_eval.evaluate()

    # Extract and return the metrics (BLEU, METEOR, ROUGE, CIDEr)
    metrics = coco_eval.eval
    return metrics

def show_images_with_captions(generated_captions, coco_annotation_file, coco_image_dir):
    """
    Display images with their generated captions.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.
        coco_image_dir (str): Path to the directory containing COCO images.
    """
    # Load the COCO dataset annotations
    coco = COCO(coco_annotation_file)

    for image_id, caption in generated_captions.items():
        # Get image information from COCO
        try:
            image_info = coco.loadImgs(image_id)[0]
            image_path = f"{coco_image_dir}/{image_info['file_name']}"
            print(f"Image path: {image_path}")  # Debug: Print the image path
        except KeyError:
            print(f"Image ID {image_id} not found in the dataset.")
            continue

        # Load and display the image using PIL and matplotlib
        try:
            image = Image.open(image_path)
        except FileNotFoundError:
            print(f"Image file not found at {image_path}")
            continue

        # Print the caption
        print(f"Generated Caption: {caption}")

        # Display the image
        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Generated Caption: {caption}", fontsize=14, wrap=True)
        plt.show()

# Generate captions for COCO validation images
generated_captions = generate_captions_for_coco(val_loader, model, vocabulary)

# Evaluate the generated captions
metrics = evaluate_captioning_model(generated_captions)

# Print evaluation metrics
print("Evaluation Metrics:", metrics)

# Display images and their captions
show_images_with_captions(generated_captions, val_annotations_path, val_images_path)


torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'motorcycle', 'parked', 'in', 'a', 'parking', 'lot', 'with', 'a', 'helmet', 'on', 'the', 'seat', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: a motorcycle parked in a parking lot with a helmet on the seat .
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'motorcycle', 'parked', 'in', 'a', 'parking', 'lot', 'with', 'a', 'helmet', 'on', 'the', 'seat', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 179765: a motorcycle parked in a parking lot with a helmet on the seat .
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
['<bos>', 'a', 'large', 'black', 'cat', 'sits', 'inside', 'of', 'an', 'empty', 'bathtub', '.', '<eos>']
type(generated_caption) = <class 'str'> , <class 'int'>
Generated Caption for Image 190236: a large black cat sits inside of an empty bathtub .
torch.Size([3, 224, 2

PTBTokenizer tokenized 2696 tokens at 46768.05 tokens per second.
PTBTokenizer tokenized 576 tokens at 13652.71 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 498, 'reflen': 469, 'guess': [498, 453, 408, 363], 'correct': [227, 71, 14, 6]}
ratio: 1.0618336886970963
Bleu_1: 0.456
Bleu_2: 0.267
Bleu_3: 0.135
Bleu_4: 0.080
computing METEOR score...
METEOR: 0.131
computing Rouge score...
ROUGE_L: 0.350
computing CIDEr score...
CIDEr: 0.262
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 814.4 ms
SPICE: 0.056
Evaluation Metrics: {'Bleu_1': 0.45582329317177545, 'Bleu_2': 0.26728730404406575, 'Bleu_3': 0.13483671952061507, 'Bleu_4': 0.07978427257672112, 'METEOR': 0.13071145719065927, 'ROUGE_L': 0.34994471888266043, 'CIDEr': 0.2622720325163794, 'SPICE': 0.05623738523704123}
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Image path: ./coco/val2017/000000179765.jpg
Generated Caption: a motorcycle parked in a parking lot with a helmet on the seat .
Image path: ./coco/val2017/000000190236.jpg
Generated Caption: a large black cat sits inside of an empty bathtub .
Image path: ./coco/val2017/000000331352.jpg
Generated Caption: a toilet with a hello kitty seat sitting in a room .
Image path: ./coco/val2017/000000517069.jpg
Generated Caption: a man is riding a bike down a road .
Image path: ./coco/val2017/000000182417.jpg
Generated Caption: a computer desk with a triple monitor computer setup .
Image path: ./coco/val2017/0

/tmp/ipykernel_12412/2048777166.py:87: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 8))


Image path: ./coco/val2017/000000117425.jpg
Generated Caption: a man with a knife and sharpener instructing a group of women .
Image path: ./coco/val2017/000000159977.jpg
Generated Caption: a giraffe standing on a dirt floor with rocks and trees in the background
Image path: ./coco/val2017/000000232538.jpg
Generated Caption: a large boat full of men is sitting on a cart
Image path: ./coco/val2017/000000177934.jpg
Generated Caption: a motorcycle parked in a parking lot with a helmet on the seat .
Image path: ./coco/val2017/000000096549.jpg
Generated Caption: a large jetliner flying over a traffic filled street .
Image path: ./coco/val2017/000000469192.jpg
Generated Caption: a large group of sheep are grazing in a field .
Image path: ./coco/val2017/000000044652.jpg
Generated Caption: a large jetliner flying over a traffic filled street .
Image path: ./coco/val2017/000000210502.jpg
Generated Caption: a large boat full of men is sitting on a cart
Image path: ./coco/val2017/000000298396.jpg